In [1]:
import tensorflow as tf

In [2]:
B = 3
D = 4
T = 5

tf.reset_default_graph()

xs = tf.placeholder(shape=[T, B, D], dtype=tf.float32)

with tf.variable_scope("MyRNN"):
    LSTMcell = tf.contrib.rnn.LSTMCell(num_units = D)
    cell = tf.contrib.rnn.MultiRNNCell([LSTMcell]) 

    output_ta = tf.TensorArray(size=T, dtype=tf.float32)
    input_ta = tf.TensorArray(size=T, dtype=tf.float32)
    input_ta = input_ta.unstack(xs)

    def body(time, output_ta_t, state):
        xt = input_ta.read(time)
        new_output, new_state = cell(xt, state)
        output_ta_t = output_ta_t.write(time, new_output)
        return (time+1, output_ta_t, new_state)

    def condition(time, output, state):
        return time < T

    time = 0
    state = cell.zero_state(B, tf.float32)

    time_final, output_ta_final, state_final = tf.while_loop(
          cond=condition,
          body=body,
          loop_vars=(time, output_ta, state))

    output_final = output_ta_final.stack()

In [8]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("output", sess.graph)
#     print(sess.run())
    writer.close()